In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
class_names = os.listdir('birds/train')
len(class_names)

500

In [4]:
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('birds/train',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1

In [5]:
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])

In [6]:
len(D)

80085

In [7]:
dataset = D
random.shuffle(dataset)

In [12]:
!pip install torchvision

  Attempting uninstall: pillow
    Found existing installation: Pillow 8.3.1
    Uninstalling Pillow-8.3.1:
      Successfully uninstalled Pillow-8.3.1


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
features = models.alexnet().features  # get the feature extraction layers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flatten = torch.nn.Flatten()# create a flatten layer
model = torch.nn.Sequential(features, flatten).to(device)  # create a new model with the flatten layer
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

In [8]:
!pip install opencv-python


In [9]:
from tqdm import tqdm
import cv2
def extract_features(train):
    features = []
    labels = []
    for i in tqdm(train,desc='loading...'):
        im = cv2.imread(i[0])
        im = im.astype('float32')
        im = cv2.resize(im, (224,224))
        im = torch.from_numpy(im)
        im = im.to(device)
        im = im.unsqueeze(0) # adding an extra dimension at index 0 which refers to the batch size =1
        im = im.permute(0, 3, 1, 2) # reorder the dimensions of the input tensor from [batch_size, height, width, num_channels] to [batch_size, num_channels, height, width]. 
        with torch.no_grad():
            outputs = model(im)
            features.append(outputs.cpu().numpy()[0])
            labels.append(i[1])
    return np.array(features), np.array(labels)

In [23]:
x_train, y_train = extract_features(dataset)

loading...: 100%|██████████| 80085/80085 [36:15<00:00, 36.80it/s]


In [24]:
x_train.shape

(80085, 9216)

In [25]:
y_train

array([ 51, 251, 149, ..., 372,  81, 424])

In [26]:
y_train.reshape(80085,1)

array([[ 51],
       [251],
       [149],
       ...,
       [372],
       [ 81],
       [424]])

In [27]:
c = np.column_stack((x_train, y_train)) 

In [28]:
c

array([[  0.        ,   0.        ,   0.        , ...,   1.97572207,
          0.81329811,  51.        ],
       [  0.        ,   0.        ,   0.        , ...,   2.78552961,
          3.68633103, 251.        ],
       [  0.        ,   0.        ,   0.        , ...,   2.92840123,
          2.74328065, 149.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   2.79152536,
          3.49096894, 372.        ],
       [  0.        ,   0.        ,   0.        , ...,   3.22269273,
          2.82211637,  81.        ],
       [  0.        ,   0.        ,   0.        , ...,   2.07398391,
          2.45444107, 424.        ]])

In [30]:
df = pd.DataFrame(c)
df

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,3.915643,2.299385,1.137324,3.923539,3.923539,3.572012,3.572012,1.975722,0.813298,51.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,3.246612,3.580022,4.209567,3.183849,3.183849,3.182083,3.182083,2.785530,3.686331,251.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,2.563519,2.723042,2.951140,2.288079,2.093834,2.191067,2.438783,2.928401,2.743281,149.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,2.138075,2.559910,3.192954,2.831107,2.461464,2.656948,1.959995,2.290751,2.970579,256.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.705342,0.358179,0.0,...,3.609416,3.637248,3.644959,3.231955,3.042504,3.054446,3.054446,3.095683,3.493525,286.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,2.608548,2.761072,5.382696,2.021265,1.948189,2.036664,2.164539,2.164539,3.451033,311.0
80081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,3.817589,3.817589,3.780623,3.219401,3.408159,3.326196,2.800683,3.441890,3.576179,254.0
80082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,3.254154,3.748947,3.748947,3.170243,2.435404,2.619712,2.702481,2.791525,3.490969,372.0
80083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,2.961524,3.222693,2.822116,2.458704,2.800992,2.967730,2.961524,3.222693,2.822116,81.0


In [31]:
# saving the dataframe
df.to_csv('bird_features.csv')

In [12]:
class_names = os.listdir('birds/test')
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('birds/test',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1
    
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])
    
    
dataset = D
random.shuffle(dataset)

In [13]:
x_test, y_test = extract_features(dataset)

loading...: 100%|██████████| 2500/2500 [01:02<00:00, 39.99it/s]


In [15]:
y_test.reshape(2500,1)

array([[424],
       [228],
       [176],
       ...,
       [248],
       [160],
       [ 46]])

In [17]:
c = np.column_stack((x_test, y_test)) 

In [19]:
df = pd.DataFrame(c)
df.to_csv('bird_test.csv')